In [9]:
library(ggplot2)
library(GenomicRanges)
library("GenomicRanges");

library("stats")
library(scales)
library(dplyr)  # Carica il pacchetto dplyr
bnoExt=function(x){
return(basename(tools::file_path_sans_ext(x)))
}
fNumber=function(x){
sapply(x,FUN=function(x){
strsplit(x,"__")[[1]][2]
})
}
# Definisci colori e tipi di linee
color_mapping <- c(
  "MEC-1_AID-KO_DMSO23" = "black",
  "MEC-1_AID-KO_Tazemetostat57" = "black",
  "MEC-1_AID-KO_Valemetostat1011" = "black",
  "AID-WT_DMSO" = "purple",
  "AID-WT_Tazemetostat" = "darkgreen",
  "AID-WT_Valemetostat" = "brown",
  "MEC-1_Idelalisib_DMSO" = "blue",
  "MEC-1_Idelalisib_Tazemetostat" = "red",
  "MEC-1_Idelalisib_Valemetostat" = "orange"
)

linetype_mapping <- c(
  "MEC-1_AID-KO_DMSO23" = "solid",
  "MEC-1_AID-KO_Tazemetostat57" = "dashed",
  "MEC-1_AID-KO_Valemetostat1011" = "dotted",
  "AID-WT_DMSO" = "solid",
  "AID-WT_Tazemetostat" = "solid",
  "AID-WT_Valemetostat" = "solid",
  "MEC-1_Idelalisib_DMSO" = "solid",
  "MEC-1_Idelalisib_Tazemetostat" = "solid",
  "MEC-1_Idelalisib_Valemetostat" = "solid"
)


In [13]:
# Carica i dati dei file BED
print("bonjovy")

tss_file <- "/sharedFolder/Data/hg38_site_TSS.bed"
tss_df <- read.table(tss_file, header=TRUE, sep="\t")

# Opzionale: ricava il centro del TSS se vuoi punto singolo
tss_df$TSS_pos <- round((tss_df$start + tss_df$end) / 2)

# Crea GRanges con punto singolo
tss_gr <- GRanges(seqnames = tss_df$chromosome,
                  ranges = IRanges(start = tss_df$TSS_pos, end = tss_df$TSS_pos),
                  strand = tss_df$strand)


all_data <- data.frame()
all_data2 <- data.frame()

for(junctions_file in list.files("/sharedFolder/Data/1_HTGTS/13_Revision1/"))
{
junctions_file=paste("/sharedFolder/Data/1_HTGTS/13_Revision1/",junctions_file,sep="")

junctions=read.table(junctions_file,header=FALSE,sep="\t")
colnames(junctions)=c("chrom", "start", "end", "junction", "score", "strand")
junctions_gr <- GRanges(seqnames = junctions$chrom, IRanges(start = junctions$start, end = junctions$end))

    
    

    
#REMOVE MYC and IGH LOCUS CRISPR Target
myc=GRanges(seqnames="chr8",ranges=IRanges(start=126735434,end=128742951))
toRemove=findOverlaps(myc,junctions_gr)@to
igh=GRanges(seqnames="chr14",ranges=IRanges(start=105550000,end=105969761))
toRemove=append(toRemove,findOverlaps(igh,junctions_gr)@to)
first=GRanges(seqnames="chrX",ranges=IRanges(start=153624154,end=153624975))
toRemove=append(toRemove,findOverlaps(first,junctions_gr)@to)
second=GRanges(seqnames="chr17",ranges=IRanges(start=18121597,end=18122666))
toRemove=append(toRemove,findOverlaps(second,junctions_gr)@to)
third=GRanges(seqnames="chr1",ranges=IRanges(start=117655795,end=117657394))
toRemove=append(toRemove,findOverlaps(third,junctions_gr)@to)
Database=read.table("/sharedFolder/Results/Hotspot/Revision1/FULL/5/DatabaseAIDINDEP.csv",header=TRUE,sep=",")
Database=Database$Category==
Database2=unique(Database[,c(1,2,3)])
for(index in seq(1,nrow(Database2))){
temp=GRanges(seqnames=Database2[index,1],ranges=IRanges(start=Database2[index,2],end=Database2[index,3]))
toRemove=append(toRemove,findOverlaps(temp,junctions_gr)@to)
}
                 
third=GRanges(seqnames="chr11",ranges=IRanges(start=114447384,end=114625636))
toRemove=append(toRemove,findOverlaps(third,junctions_gr)@to)

    
    
junctions_gr=junctions_gr[-toRemove,]
#############################
#TEMP IDELALISIB HOTSPOT
toKeep=c()
    Database=read.table("/sharedFolder/Results/Hotspot/Revision1/FULL/5/Database.csv",header=TRUE,sep=",")
Database2=Database[grep("Idelalisib",Database$HotspotBelonging),c(1,2,3)]

for(index in seq(1,nrow(Database2))){
temp=GRanges(seqnames=Database2[index,1],ranges=IRanges(start=Database2[index,2],end=Database2[index,3]))
toKeep=append(toKeep,findOverlaps(temp,junctions_gr)@to)
}    
junctions_gr=junctions_gr[toKeep,]
    
   
    

distance_GR=distanceToNearest(junctions_gr,tss_gr)
distance=start(junctions_gr[queryHits(distance_GR)])- start(tss_gr[subjectHits(distance_GR)])


rpm=log2(countOverlaps(junctions_gr[queryHits(distance_GR)],promoters(junctions_gr,upstream=1000,downstream=1000))+1)
    james=rpm
f=smooth.spline(distance,rpm,spar=0.6)
############################
distanze=distance
intervalli <- seq(floor(min(distanze)), ceiling(max(distanze)), by = 10000)
conteggio_intervallo <- cut(distanze, breaks = intervalli,labels=intervalli[-1])
conteggio_df <- data.frame(Intervallo = as.numeric(levels(conteggio_intervallo)),
                           Frequenza = table(conteggio_intervallo))
conteggio_df=conteggio_df[,-2]
#f2=smooth.spline(conteggio_df[,1],conteggio_df[,2],spar=0)    #SPLINE
loess_fit <- loess(conteggio_df[,2] ~ conteggio_df[,1])
loess_values <- predict(loess_fit)
f2=cbind(conteggio_df[,1],loess_values)
colnames(f2)=c("x","y")
f2=data.frame(f2)    
#############################
cycle_data <- data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file))))#
cycle_data2 <- data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file))))#  SPLINE
#cycle_data2 <- data.frame(distance = conteggio_df[,1], counts = conteggio_df[,2], cycle = fNumber(bnoExt(basename(junctions_file))))# RAW
    
    
all_data <- rbind(all_data, cycle_data)
all_data2 <- rbind(all_data2, cycle_data2)

}
all_data=as.data.frame(all_data)
plot=ggplot(all_data, aes(x = distance, y = rpm, color = cycle, linetype = cycle)) +
geom_line(data = all_data, aes(x = distance, y = rpm)) + #
scale_color_manual(values = color_mapping) +
scale_linetype_manual(values = linetype_mapping) +
labs(x = "Distance from TSS", y = "Spline fitted junctions distribution", title = "Spline fitted junctions distribution vs TSS distance") +
  theme_minimal() +
  theme(legend.position = "top") +
  coord_cartesian(xlim = c(-10000,10000)) +
  theme(panel.grid = element_blank())  # Rimuovi la griglia


ggsave("/sharedFolder/Results/Hotspot/Revision1/FULL/TSS_IDE.pdf", plot, bg = "white")



# Definisci gli intervalli
#ggplot(all_data2, aes(x = distance, y = counts, color = cycle, linetype = cycle)) +
#geom_line(data = all_data2, aes(x = distance, y = counts)) + #
#scale_color_manual(values = color_mapping) +
#scale_linetype_manual(values = linetype_mapping) +
#labs(x = "Distance from TSS", y = "log10(Counts) 2kb", title = "log10(Counts) (2kb) vs TSS distance") +
#  theme_minimal() +
#  theme(legend.position = "top") +
#  coord_cartesian(xlim = c(-10000,10000))#

[1] "bonjovy"


Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f$x, rpm = f$y, cycle = fNumber(bnoExt(basename(junctions_file)))):
“row names were found from a short variable and have been discarded”
Warning message in data.frame(distance = f2$x, counts = f2$y, cycle = fNumber(bnoExt(basename(juncti

In [5]:
head(all_data)

,distance,rpm,cycle
,<dbl>,<dbl>,<chr>
1,-1513450,0.9593467,AID-WT_DMSO
2,-1438327,1.0485121,AID-WT_DMSO
3,-474040,1.1273262,AID-WT_DMSO
4,-472755,1.1268856,AID-WT_DMSO
5,-472362,1.1267509,AID-WT_DMSO
6,-470106,1.1259786,AID-WT_DMSO


In [6]:
unique(all_data$cycle)

[1] "AID-WT_DMSO"                         
[2] "AID-WT_Tazemetostat"                 
[3] "AID-WT_Valemetostat"                 
[4] "MEC-1_AID-KO_DMSO23"                 
[5] "MEC-1_AID-KO_Tazemetostat57"         
[6] "MEC-1_AID-KO_Valemetostat1011"       
[7] "MEC-1_AID-WT_Idelalisib_DMSO"        
[8] "MEC-1_AID-WT_Idelalisib_Tazemetostat"
[9] "MEC-1_AID-WT_Idelalisib_Valemetostat"